In [19]:
import pandas as pd
import numpy as np
from collections import Counter

# pd.options.display.float_format = "{:,.4f}".format

# import matplotlib.pyplot as plt
# import seaborn as sns

# import statsmodels.api as sm
# from scipy.stats import norm
# from sklearn.linear_model import LinearRegression
# from arch import arch_model

# import warnings
# warnings.filterwarnings("ignore")

# import importlib
# import utils as ut
# importlib.reload(ut)

--- Day 1: Historian Hysteria ---

In [18]:
file_path = './data/input_1.txt'
data = np.loadtxt(file_path, dtype=int)

left, right = np.sort(data[:, 0]), np.sort(data[:, 1])

total_distance = np.sum(np.abs(left - right))
print("Total Distance:", total_distance)

Total Distance: 2086478


In [23]:
counter = Counter(right)
sum(counter[l] * l for l in left)

np.int64(24941624)

--- Day 2: Red-Nosed Reports ---